In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers,initializers
from keras.layers import GaussianNoise
import keras.backend as Kb
import keras.losses

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [5]:
### Basis functions

num_basis = [3**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

### Train Test split

In [6]:
### pre-processing (multiply with lmc estimates)

phi1 = 1.28*phi + 0.36*phi   ### variable 1 basis functions
phi2 = 0.639*phi + 0.903*phi ### variable 2 basis functions

In [45]:
## Functions for calculation of MSE and MAE
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

mse_var1 = []
mse_var2 = []
correlation = []

## Independent DeepKriging with replicates

In [23]:
## Functions for calculation of MSE and MAE
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

mse_var1 = []
mse_var2 = []
for i in range(1):
    df_loc = pd.read_csv("../synthetic_data_simulations/2d_gaussian_1200_"+str(i+1)+".csv", sep = ",")

    N = len(df_loc)
    s = np.vstack((df_loc["x"],df_loc["y"])).T
    y = np.array(df_loc[["var1","var2"]])
    num_basis = [2**2,5**2,9**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    #knots_1d = [np.linspace(0,1,i) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))
    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                    test_size=0.3333)
    N_train = s_train.shape[0]
    N_test = s_test.shape[0]


    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim = encoder_train.shape[1],  
                kernel_initializer=initializers.RandomNormal(stddev=0.01), activation='relu'))
    #     model.add(Dense(100, input_dim = encoder_train.shape[1],  kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
    #     model.add(Dense(100, activation='relu'))
    model.add(Dense(100, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                bias_regularizer=regularizers.l2(1e-4),
                activity_regularizer=regularizers.l2(1e-5),activation='relu'))
    #     model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    #     model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    #model.add(Dropout(rate=0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(1, activation='linear'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae','mse'])

    result = model.fit(encoder_train, y_train[:,0], 
                       validation_data=(encoder_test,y_test[:,0]), epochs = 300, batch_size = 64, verbose = 2)

    callbacks = [EarlyStopping(monitor='val_loss', patience=40),
                 ModelCheckpoint(filepath='Biv_gaussian.h5', monitor='val_loss', save_best_only=True)]
    result = model.fit(encoder_train, y_train[:,0], callbacks=callbacks, 
                       validation_data=(encoder_test,y_test[:,0]), epochs = 250, batch_size = 64, verbose = 2)
    model = keras.models.load_model('Biv_gaussian.h5')
    y_pred1 = model.predict(encoder_test)

#     model = Sequential()
#     # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
#     model.add(Dense(100, input_dim = encoder_train.shape[1],  
#                 kernel_initializer=initializers.RandomNormal(stddev=0.01), activation='relu'))
#     #     model.add(Dense(100, input_dim = encoder_train.shape[1],  kernel_initializer='he_uniform', activation='relu'))
#     # model.add(Dropout(rate=0.5))
#     # model.add(BatchNormalization())
#     #     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
#                 bias_regularizer=regularizers.l2(1e-4),
#                 activity_regularizer=regularizers.l2(1e-5),activation='relu'))
#     #     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     #     model.add(Dense(100, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     # model.add(Dense(100, activation='relu'))
#     #model.add(Dropout(rate=0.5))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     #model.add(BatchNormalization())
#     model.add(Dense(1, activation='linear'))
#     NB_START_EPOCHS = 50 
#     # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
#     BATCH_SIZE = 64  
#     fold_no = 1
#     optimizer = keras.optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimizer, loss='mse', metrics=['mae','mse'])

#     result = model.fit(encoder_train, y_train[:,1], 
#                        validation_data=(encoder_test,y_test[:,1]), epochs = 300, batch_size = 64, verbose = 2)

#     callbacks = [EarlyStopping(monitor='val_loss', patience=40),
#                  ModelCheckpoint(filepath='Biv_gaussian.h5', monitor='val_loss', save_best_only=True)]
#     result = model.fit(encoder_train, y_train[:,1], callbacks=callbacks, 
#                        validation_data=(encoder_test,y_test[:,1]), epochs = 250, batch_size = 64, verbose = 2)
# #     model = keras.models.load_model('Biv_gaussian.h5')
#     y_pred2 = model.predict(encoder_test)


    # Mean Squared Error
    mse_var1.append(mse(y_pred1, y_test[:,0]))
#     mse_var2.append(mse(y_pred2, y_test[:,1]))

Epoch 1/300
13/13 - 1s - loss: 1.3415 - mae: 0.6128 - mse: 1.3234 - val_loss: 0.8060 - val_mae: 0.5588 - val_mse: 0.7886
Epoch 2/300
13/13 - 0s - loss: 0.8211 - mae: 0.5110 - mse: 0.8039 - val_loss: 0.3845 - val_mae: 0.3778 - val_mse: 0.3676
Epoch 3/300
13/13 - 0s - loss: 0.5364 - mae: 0.4277 - mse: 0.5196 - val_loss: 0.2402 - val_mae: 0.2909 - val_mse: 0.2236
Epoch 4/300
13/13 - 0s - loss: 0.3342 - mae: 0.3401 - mse: 0.3178 - val_loss: 0.1720 - val_mae: 0.2538 - val_mse: 0.1557
Epoch 5/300
13/13 - 0s - loss: 0.2373 - mae: 0.2713 - mse: 0.2211 - val_loss: 0.1552 - val_mae: 0.2287 - val_mse: 0.1392
Epoch 6/300
13/13 - 0s - loss: 0.1805 - mae: 0.2426 - mse: 0.1646 - val_loss: 0.1613 - val_mae: 0.2449 - val_mse: 0.1455
Epoch 7/300
13/13 - 0s - loss: 0.1422 - mae: 0.2378 - mse: 0.1265 - val_loss: 0.1524 - val_mae: 0.2162 - val_mse: 0.1368
Epoch 8/300
13/13 - 0s - loss: 0.1260 - mae: 0.2140 - mse: 0.1105 - val_loss: 0.1456 - val_mae: 0.2216 - val_mse: 0.1303
Epoch 9/300
13/13 - 0s - loss: 0

KeyboardInterrupt: 

In [21]:
mse_var1

[1.6596804632087576]

In [19]:
y_pred1[:,1]


IndexError: index 1 is out of bounds for axis 1 with size 1

In [17]:
y_test

array([[-1.35192119e+00, -1.11566737e-01],
       [ 3.24607966e-01,  9.10878497e-01],
       [-2.60916552e+00,  1.19471810e+00],
       [-1.29854368e+00,  2.72833636e+00],
       [-6.36592760e-01,  7.09306462e-01],
       [-1.72179499e-02,  5.71126932e-01],
       [ 4.70802421e-02,  3.67449608e-01],
       [-5.02442459e-01,  2.19964194e-01],
       [-2.10049470e-01,  1.57944606e-01],
       [ 1.60019299e-02,  3.53711780e-01],
       [ 1.13474809e-01,  4.35573618e+00],
       [-4.62000323e-01,  4.57175542e+00],
       [ 1.23258357e+00,  1.00112497e+00],
       [-5.43343646e-01, -1.04736761e-01],
       [-1.54091296e-01,  3.82301367e-01],
       [-2.79138565e-01,  1.14270739e+00],
       [-5.48104288e-02,  3.40096470e+00],
       [-2.71982777e+00,  9.30134428e-01],
       [ 2.35339826e-02,  4.22769062e+00],
       [-3.97782934e-01,  1.07433095e+00],
       [-1.37552085e+00, -3.59828608e-01],
       [-4.12751149e-01, -5.01662932e-02],
       [-2.87065727e-01,  1.15146103e+00],
       [-1.

In [27]:
### estimate for LMC
## Functions for calculation of MSE and MAE
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

mse_var1 = []
mse_var2 = []
for i in range(1):
    df_loc = pd.read_csv("../synthetic_data_simulations_gaussian/2d_gaussian_1200_"+str(i+1)+".csv", sep = ",")

    N = len(df_loc)
    s = np.vstack((df_loc["x"],df_loc["y"])).T
    y = np.array(df_loc[["var1","var2"]])
    num_basis = [2**2,5**2,9**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    #knots_1d = [np.linspace(0,1,i) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))
    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
   ### pre-processing (multiply with lmc estimates)

    phi1 = 1.28*phi + 0.36*phi   ### variable 1 basis functions
    phi2 = 0.639*phi + 0.903*phi ### variable 2 basis functions
    N_train = s_train.shape[0]
    N_test = s_test.shape[0]
    s_train, s_test,x_train1,x_test1,x_train2,x_test2,y_train, y_test= train_test_split(s, phi1, phi2, y, 
                                                            test_size=0.333)
#     data_train = np.hstack((encoder_train,y_train))
#     n_rows = data_train.shape[0]
#     random_indices = np.random.choice(n_rows, size=10000, replace=True)
#     resampled_data_train = data_train[random_indices, :]
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(1, input_dim = x_train1.shape[1],  
                    kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
    #model.add(Dense(50, activation='relu'))
    #model.add(Dropout(rate=0.5))
#     model.add(Dense(10, activation='relu'))
    #model.add(BatchNormalization())
#     model.add(Dense(1, activation='linear'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

    callbacks = [EarlyStopping(monitor='val_loss', patience=100),
             ModelCheckpoint(filepath='Biv_nonStationary_model.h5', monitor='val_loss', save_best_only=True)]
    result = model.fit(x_train1, y_train[:,0],callbacks = callbacks, 
                       validation_data=(x_test1,y_test[:,0]), epochs = 400, batch_size = 128, verbose = 1)
    
    
    

#     result = model.fit(x_train, y_train, callbacks=callbacks, 
#                        validation_data=(x_test,y_test), epochs = 200, batch_size = 64, verbose = 2)
    model = keras.models.load_model('Biv_nonStationary_model.h5')
    y_pred1 = model.predict(x_test1)
    mse_var1.append(mse(y_test[:,0], y_pred1))
    
    
#     data_train = np.hstack((encoder_train,y_train))
#     n_rows = data_train.shape[0]
#     random_indices = np.random.choice(n_rows, size=10000, replace=True)
#     resampled_data_train = data_train[random_indices, :]
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(1, input_dim = x_train2.shape[1],  
                    kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
    #model.add(Dense(50, activation='relu'))
    #model.add(Dropout(rate=0.5))
#     model.add(Dense(10, activation='relu'))
    #model.add(BatchNormalization())
#     model.add(Dense(1, activation='linear'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

    callbacks = [EarlyStopping(monitor='val_loss', patience=100),
             ModelCheckpoint(filepath='Biv_nonStationary_model.h5', monitor='val_loss', save_best_only=True)]
    result = model.fit(x_train2, y_train[:,1],callbacks = callbacks, 
                       validation_data=(x_test2,y_test[:,1]), epochs = 400, batch_size = 128, verbose = 2)
    
    
    

#     result = model.fit(x_train, y_train, callbacks=callbacks, 
#                        validation_data=(x_test,y_test), epochs = 200, batch_size = 64, verbose = 2)
    model = keras.models.load_model('Biv_nonStationary_model.h5')
    y_pred2 = model.predict(x_test2)
    
    
    

    # Mean Squared Error
    mse_var2.append(mse(y_test[:,1], y_pred2))
#     mse_var2.append(mean_squared_error(y_test[:,1], y_pred[:,1]))
#     print(y_pred1[:,0])
#     print(y_pred2[:,0])
    correlation.append(np.corrcoef(np.array(y_pred1[:,0]),np.array(y_pred2[:,0]))[0,1])
    print(i)

Epoch 1/400
7/7 [==============================] - 1s 70ms/step - loss: 1.6128 - mse: 1.6128 - mae: 0.7477 - val_loss: 1.6454 - val_mse: 1.6454 - val_mae: 0.7559
Epoch 2/400
7/7 [==============================] - 0s 37ms/step - loss: 1.4350 - mse: 1.4350 - mae: 0.7144 - val_loss: 1.6190 - val_mse: 1.6190 - val_mae: 0.7407
Epoch 3/400
7/7 [==============================] - 0s 31ms/step - loss: 1.1906 - mse: 1.1906 - mae: 0.6656 - val_loss: 1.5975 - val_mse: 1.5975 - val_mae: 0.7270
Epoch 4/400
7/7 [==============================] - 0s 37ms/step - loss: 1.1835 - mse: 1.1835 - mae: 0.6675 - val_loss: 1.5794 - val_mse: 1.5794 - val_mae: 0.7149
Epoch 5/400
7/7 [==============================] - 0s 30ms/step - loss: 1.4563 - mse: 1.4563 - mae: 0.6846 - val_loss: 1.5640 - val_mse: 1.5640 - val_mae: 0.7043
Epoch 6/400
7/7 [==============================] - 0s 31ms/step - loss: 1.3553 - mse: 1.3553 - mae: 0.6580 - val_loss: 1.5510 - val_mse: 1.5510 - val_mae: 0.6954
Epoch 7/400
7/7 [===========

7/7 [==============================] - 0s 29ms/step - loss: 1.1016 - mse: 1.1016 - mae: 0.5701 - val_loss: 1.4632 - val_mse: 1.4632 - val_mae: 0.6194
Epoch 102/400
7/7 [==============================] - 0s 29ms/step - loss: 1.3226 - mse: 1.3226 - mae: 0.5935 - val_loss: 1.4631 - val_mse: 1.4631 - val_mae: 0.6190
Epoch 103/400
7/7 [==============================] - 0s 28ms/step - loss: 1.1621 - mse: 1.1621 - mae: 0.5558 - val_loss: 1.4630 - val_mse: 1.4630 - val_mae: 0.6188
Epoch 104/400
7/7 [==============================] - 0s 31ms/step - loss: 1.2971 - mse: 1.2971 - mae: 0.5985 - val_loss: 1.4629 - val_mse: 1.4629 - val_mae: 0.6186
Epoch 105/400
7/7 [==============================] - 0s 31ms/step - loss: 1.2451 - mse: 1.2451 - mae: 0.5823 - val_loss: 1.4627 - val_mse: 1.4627 - val_mae: 0.6184
Epoch 106/400
7/7 [==============================] - 0s 30ms/step - loss: 1.3420 - mse: 1.3420 - mae: 0.5987 - val_loss: 1.4625 - val_mse: 1.4625 - val_mae: 0.6182
Epoch 107/400
7/7 [===========

7/7 [==============================] - 0s 27ms/step - loss: 1.3380 - mse: 1.3380 - mae: 0.5673 - val_loss: 1.4606 - val_mse: 1.4606 - val_mae: 0.6156
Epoch 152/400
7/7 [==============================] - 0s 29ms/step - loss: 1.0869 - mse: 1.0869 - mae: 0.5489 - val_loss: 1.4605 - val_mse: 1.4605 - val_mae: 0.6156
Epoch 153/400
7/7 [==============================] - 0s 33ms/step - loss: 1.0693 - mse: 1.0693 - mae: 0.5505 - val_loss: 1.4605 - val_mse: 1.4605 - val_mae: 0.6156
Epoch 154/400
7/7 [==============================] - 0s 37ms/step - loss: 1.2614 - mse: 1.2614 - mae: 0.5813 - val_loss: 1.4605 - val_mse: 1.4605 - val_mae: 0.6156
Epoch 155/400
7/7 [==============================] - 0s 28ms/step - loss: 1.2485 - mse: 1.2485 - mae: 0.5597 - val_loss: 1.4605 - val_mse: 1.4605 - val_mae: 0.6156
Epoch 156/400
7/7 [==============================] - 0s 21ms/step - loss: 1.0803 - mse: 1.0803 - mae: 0.5527 - val_loss: 1.4604 - val_mse: 1.4604 - val_mae: 0.6155
Epoch 157/400
7/7 [===========

7/7 [==============================] - 0s 35ms/step - loss: 1.5034 - mse: 1.5034 - mae: 0.5976 - val_loss: 1.4598 - val_mse: 1.4598 - val_mae: 0.6158
Epoch 202/400
7/7 [==============================] - 0s 25ms/step - loss: 1.0868 - mse: 1.0868 - mae: 0.5286 - val_loss: 1.4599 - val_mse: 1.4599 - val_mae: 0.6158
Epoch 203/400
7/7 [==============================] - 0s 28ms/step - loss: 1.2678 - mse: 1.2678 - mae: 0.5690 - val_loss: 1.4600 - val_mse: 1.4600 - val_mae: 0.6159
Epoch 204/400
7/7 [==============================] - 0s 26ms/step - loss: 1.2389 - mse: 1.2389 - mae: 0.5673 - val_loss: 1.4601 - val_mse: 1.4601 - val_mae: 0.6160
Epoch 205/400
7/7 [==============================] - 0s 28ms/step - loss: 1.2141 - mse: 1.2141 - mae: 0.5638 - val_loss: 1.4602 - val_mse: 1.4602 - val_mae: 0.6161
Epoch 206/400
7/7 [==============================] - 0s 30ms/step - loss: 1.3594 - mse: 1.3594 - mae: 0.6036 - val_loss: 1.4602 - val_mse: 1.4602 - val_mae: 0.6160
Epoch 207/400
7/7 [===========

KeyboardInterrupt: 

In [29]:
mse_var2

[]

In [37]:
### lmc coefficients multiplied 
print("mean var 1 : ",np.mean(mse_var1))
print("mean var 2 : ",np.mean(mse_var2))
print("variance var 1 : ",np.var(mse_var1))
print("variance var 2 : ",np.var(mse_var2))
print("correlation : ",np.mean(correlation))

mean var 1 :  0.0073751523257748084
mean var 2 :  0.00571064356234115
variance var 1 :  5.973672588415026e-07
variance var 2 :  3.679894352165365e-07
correlation :  0.8773736657612876


In [41]:
### independent single variable training
print("mean var 1 : ",np.mean(mse_var1))
print("mean var 2 : ",np.mean(mse_var2))
print("variance var 1 : ",np.var(mse_var1))
print("variance var 2 : ",np.var(mse_var2))
print("correlation : ",np.mean(correlation))

mean var 1 :  0.0069509933472384116
mean var 2 :  0.0055481564103758065
variance var 1 :  2.131914925257068e-07
variance var 2 :  3.657565294016452e-07
correlation :  0.8698024609977338


In [47]:
### low rank approximation of lmc
print("mean var 1 : ",np.mean(mse_var1))
print("mean var 2 : ",np.mean(mse_var2))
print("variance var 1 : ",np.var(mse_var1))
print("variance var 2 : ",np.var(mse_var2))
print("correlation : ",np.mean(correlation))

mean var 1 :  0.1283436222252356
mean var 2 :  0.08093607664785937
variance var 1 :  0.001727034795057943
variance var 2 :  0.0008055564705659079
correlation :  0.8722977348119214


## comparison results 

### Biv_deepKriging
- mean var 1 :  0.006534991334629507
- mean var 2 :  0.005235456543853871
- variance var 1 :  4.2874083303066245e-07
- variance var 2 :  1.785717711119858e-07
- correlation :  0.8730484697671853

### lmc coefficients multiplied 
- mean var 1 :  0.0073751523257748084 
- mean var 2 :  0.00571064356234115
- variance var 1 :  5.973672588415026e-07
- variance var 2 :  3.679894352165365e-07
- correlation :  0.8773736657612876

### Independent single variable training
- mean var 1 :  0.0069509933472384116
- mean var 2 :  0.0055481564103758065
- variance var 1 :  2.131914925257068e-07
- variance var 2 :  3.657565294016452e-07
- correlation :  0.8698024609977338

### low rank approximation of lmc
- mean var 1 :  0.1283436222252356
- mean var 2 :  0.08093607664785937
- variance var 1 :  0.001727034795057943
- variance var 2 :  0.0008055564705659079
- correlation :  0.8722977348119214

In [49]:
(0.006144+0.005206)/2


0.0056749999999999995

In [ ]:
start_time = time.time()
for i in range(1):
    s_train, s_test,x_train,x_test,y_train, y_test= train_test_split(s, phi, y, 
                                                            test_size=0.05)
#     data_train = np.hstack((encoder_train,y_train))
#     n_rows = data_train.shape[0]
#     random_indices = np.random.choice(n_rows, size=10000, replace=True)
#     resampled_data_train = data_train[random_indices, :]
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim = x_train.shape[1],  
                    kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    #model.add(Dense(50, activation='relu'))
    #model.add(Dropout(rate=0.5))
#     model.add(Dense(10, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(2, activation='linear'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

    callbacks = [EarlyStopping(monitor='val_loss', patience=100),
             ModelCheckpoint(filepath='Biv_nonStationary_model.h5', monitor='val_loss', save_best_only=True)]
    result = model.fit(x_train, y_train,callbacks = callbacks, 
                       validation_data=(x_test,y_test), epochs = 400, batch_size = 128, verbose = 2)
    
    
    

#     result = model.fit(x_train, y_train, callbacks=callbacks, 
#                        validation_data=(x_test,y_test), epochs = 200, batch_size = 64, verbose = 2)
    model = keras.models.load_model('Biv_nonStationary_model.h5')
    y_pred = model.predict(x_test)

    # Mean Squared Error
    mse_var1.append(mean_squared_error(y_test[:,0], y_pred[:,0]))
    mse_var2.append(mean_squared_error(y_test[:,1], y_pred[:,1]))
end_time = time.time()
print("%s seconds", end_time - start_time)